# Rebalance - example

This is the analytical procedure to rebalance a portfolio. 
The computations must be done after `Dfix` closing and before the rebalance execution in the `Droll` (usually next business day after `Dfix`).

Let's assume that we already have decided to follow a portfolio strategy with the following parameters:
1. mixture of CVaR's as a risk measure with the following parameters: 
    - confidence levels `alpha = [0.95, 0.90, 0.85]`
    - mixture coefficients `coef = [0.2, 0.3, 0.5]`
2. Sharpe optimization strategy (*i.e.* maximization of mCVaR-Sharpe ratio, `rtype='Sharpe'`)
3. Quarterly rebalanced, `freq='Q'`
4. With the length of the historical data used in the calibration of the weights of `hlength=3.25` years.

>Note: The workflow is the same if other portfolio strategy was adopted. 

We start by importing the **azapy** package as well as other useful packages.

In [1]:
import pandas as pd

import sys
sys.path.append("..")
import azapy as az

### Collect the relevant historical market data

- `symb` is the list of stock symbols (portfolio components).
- `sdate` and `edate` are the start and end dates of historical time-series.
- `mktdir` is the name of the directory used as a buffer for market data collected from the data provider *alphavantage*.
    
> Note: if the flag `force=False` then a reading from `dir=mktdir` is attempted. If it fails then the data provider servers will be accessed. The new data will be saved to the `dir=mktdir`. *For more information see the azapy.readMkT function documentation.*

> Hint: if you want to overwrite existing old data in directory `mktdir` then set `force=True`.

In [2]:
symb = ['GLD', 'TLT', 'XLV', 'VGT', 'PSJ']

sdate = pd.to_datetime("2012-01-01")
edate = pd.to_datetime("today")

mktdir = "./MkTdata"

mktdata = az.readMkT(symb, dstart = sdate, dend = edate, dir=mktdir, force=False) 

Read GLD form ./MkTdata
Read TLT form ./MkTdata
Read XLV form ./MkTdata
Read VGT form ./MkTdata
Read PSJ form ./MkTdata


### Check the market data

At this point it is a good idea to check the market data quality. 
The **azapy** function `summary_MktData` my come in handy. It returns:

- `symbol` the symbols included in `mktdata`. These are the portfolio components. 
- `begin` is the beginning of the historical time-series for each symbol.
- `end` is the end date of the historical time series (the most recent date).
- `length` is history length in number of business days.
- `na_total` is the total number of missing values, `nan`.
- `na_b` is the total number of missing records at the beginning of the time-series (in number of business days).
- `na_e` is the total number of missing records at the end of the time-series (in number of business days).
- `cont` is the total number of missing records (at the beginning, end and in the middle of the time-series). 

We are looking for:
- `begin` date to be prior `end` date minus `length`. In our case this condition is overwhelmingly satisfied.
- `end` must be equal to `Dfix`. Therefore it is very important to retrieve the market data in the `Dfix` date after the market closing and after the data provider had updated their database. 
- ideally `na_total`, `na_b`, `na_e` and `cont` should be `0`. Missing data could be a serious problem since in general gaps should be filled before proceeding further. 

In [3]:
az.summary_MkTData(mktdata)

,symbol,begin,end,length,na_total,na_b,na_e,cont
0,GLD,2012-01-03,2021-08-12,2419,0,0,0,0
1,PSJ,2012-01-03,2021-08-12,2419,0,0,0,0
2,TLT,2012-01-03,2021-08-12,2419,0,0,0,0
3,VGT,2012-01-03,2021-08-12,2419,0,0,0,0
4,XLV,2012-01-03,2021-08-12,2419,0,0,0,0


### Set the dispersion measure parameters 

For us they are the confidence levels and mixture coefficients mentioned before.

In [4]:
alpha = [0.95, 0.90, 0.85]
coef = [0.2, 0.3, 0.5]

### Set up the CVaRAnalyzer class

In this class, `hlength` and `freq` default values are `3.25` and `'Q'`, respectively *(see the documentation)*. Here we set them explicitly only as an example.

In [5]:
cr1 = az.CVaRAnalyzer(alpha, coef, mktdata, hlength=3.25, freq='Q')

### Set up the existing portfolio positions

- `ns` is the existing number of shares per asset. These must be greater or equal to 0. Our portfolio strategies do not allowed for short positions. *See also the documentation* for `getPositions` method for additional information.
- `cash` is the additional cash in dollars that we what to add to the total investment capital. A negative value means a capital reduction. In our example we retire a portion of the invested capital (*i.e.* 3000 dollars).

In [6]:
ns = pd.Series([300, 40, 150, 50, 50], index=symb)
cash = -3000

### Get the rebalancing positions

Inputs:

- `rtype`is the optimization strategy. In our case it is set to `'Sharpe'`.
- `mu` is the risk free rate accessible to us. That is 0. *See also the documentation* for additional information.
- `nshares` is the pd.Series containing the existing number of shares per asset.
- `cash` is the additional amount of cash to be added to the capital.

Returned information:
- `old_nsh` column contains the exiting number of shares per asset as well as the additional cash. These are the input values.
- `new_nsh` column contains the new numbers of shares per asset. The `cash` entry is the amount of cash that must be rolled to the next rebalancing event. It is generated by rounding the number of shares to integer values. Although its value is small, it could be positive or negative. It is the investor responsibility to compensate for a capital shortfall. 
- `diff_nsh` it is the actual number of shares that need to be sold and both in order to rebalance the portfolio. Under the `cash` entry is the actual amount of cash that is hold by the investor after the rebalance. In our case we have asked to retire 3000 dollars. However, after rebalance, the new position assumes a capital shortfall of roughly 47 dollars. Therefore we are left with only 2953 dollars.
- `weights` column holds the portfolio weights in algebraic form. Under the `cash` entry we have the total value of the shares at the closing price in the `Dfix` date. This may be different than the actual value recorder at the transaction time in the `Droll` date (usually the next business day).
- `price` the shares price at the `Difx` closing.

> Note: The rebalancing positions are computed as of `Dfix` date. However, the execution price in the `Droll` date could be bigger or smaller than the closing price in the `Difx` date. Therefore, the invested capital could be different than the one presented in this report. It is the investor responsibility to cover for the potential capital shortfall. 

In [7]:
cr1.getPositions(mu=0., rtype='Sharpe', nshares=ns, cash=cash)

,old_nsh,new_nsh,diff_nsh,weights,prices
GLD,300,309.00,9.00,0.504169,164.04
PSJ,50,35.00,-15.00,0.054494,155.91
TLT,40,47.00,7.00,0.067807,146.24
VGT,50,44.00,-6.00,0.182096,414.20
XLV,150,145.00,-5.00,0.191433,132.86
cash,-3000,-11.89,2988.11,100507.990000,1.00
